In [2]:
from yahoofinancials import YahooFinancials
import numpy as np

In [105]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_BV(self, numofYears=20):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
        
    def get_BV_GR(self, bv):
        return [np.size(bv)-1, (bv[0] - bv[-1])/abs(bv[-1]*(np.size(bv)-1))]
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def get_PE(self):
        print(self._stock_summary_data('trailingPE'))
        print(self._stock_summary_data('forwardPE'))
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        print(suggestPrice)
        print(stockprice)
        if suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'
            
            
        

In [110]:
x = cookFinancials('GOOGL')

In [111]:
bv = x.get_BV(20)

In [112]:
bv.insert(0, x.get_book_value())
print(bv)

[212920000000, 201442000000, 177628000000, 152502000000, 139036000000]


In [113]:
bvgr = x.get_BV_GR(bv)
print(bvgr)

[4, 0.1328504847665353]


In [114]:
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 8;
rRate = 0.2;
safty = 0.5
PE = x.get_PE()

35.161346
29.689508


In [115]:
x.get_pe_ratio()
x.get_PE()

35.161346
29.689508


32.425427

In [103]:
print(cEPS)
print(growth)
print(years)
print(rRate)
print(PE)
print(safty)

8.777999242122782
0.24722522552961448
8
0.2
30.015818000000003
0.5


In [104]:
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
print(price)

(358.8019892120326, 179.4009946060163)


In [107]:
x.get_decision(price[1],x.get_current_price())

179.4009946060163
286.815


'do not buy'